In [1]:
# need to add dataset to a folder in google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
#%tensorflow_version 2.x

In [3]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import SimpleITK as sitk
import pickle
from tensorflow import saved_model as sm

In [4]:
def import_data_numpy(path):
    """
    Imports png images and returns as a list of tensors to be used in training
    :param path: path to directory where images are
    :return: list of Tensors
    """
    tensor_list = []
    for file in os.listdir(path):
        if file.endswith(".png") or file.endswith(".tif"):
            # Converts to RGB because the vessel images are black and white
            png_img = Image.open(os.path.join(path, file)).convert('RGB')
            # Encodes images as tensors and adds to the list
            tensor_list.append(np.array(png_img.getdata()).reshape((128,128,3)))
        # Returns list of tensors to be used in model
    tensor_list = np.asarray(tensor_list)
    return tensor_list


In [5]:
def _encoder_block_unet(input, num_filters):
    """
    Encoder logic for U-Net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.MaxPool2D((2, 2))(x)
    # x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x


def _encoder_block_kinet(input, num_filters):
    """
    Encoder logic for Ki-Net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.UpSampling2D(size=(2, 2), interpolation="bilinear")(x)
    # x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    p = layers.MaxPool2D((2, 2))(x)
    return x


def _decoder_block_unet(input, num_filters):
    """
    Decoder logic for U-Net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.UpSampling2D(size=(2, 2), interpolation="bilinear")(x)
    # x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x


def _decoder_block_kinet(input, num_filters):
    """
    Decoder logic for Ki-net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.MaxPool2D((2, 2))(x)
    # x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    print(type(x))
    return x


def _crfb(x1, x2, num_filters, scale_factor):
    """
    CRFB is the cross residual fusion block which fuses the outputs at the specified layers
    from one model to the other, and returns the results as input for the next step.
    :param x1: KerasTensor
    :param x2: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :param scale_factor: Float - scale factor for resizing (upsampling/downsampling) of output
    :return: KerasTensor
    """
    new_w = int(x2.shape[1] * scale_factor)
    new_h = int(x2.shape[2] * scale_factor)
    out = layers.Conv2D(num_filters, 3, padding="same")(x2)
    out = layers.Activation("relu")(out)
    out = tf.image.resize(out, [new_h, new_w])
    output = layers.Concatenate()([x1, out])
    return output

In [6]:
def kiunet(input_shape):
    """
    Architecture of the KiU-Net model, which combines the Ki-Net and U-Net architecture
    :param input_shape: Tuple of the shape of the input
    :return: keras.Model
    """
    inputs = layers.Input(shape=input_shape)

    # ENCODER BLOCK #

    s1 = _encoder_block_unet(inputs, 32)  # U NET ENCODER
    k1 = _encoder_block_kinet(inputs, 32)  # KINET ENCODER

    u1 = _crfb(s1, k1, 32, 0.25)  # CRFB U1 UNET
    o1 = _crfb(k1, s1, 32, 4)  # CRFB O1 KINET

    s2 = _encoder_block_unet(u1, 64)  # UNET ENCODER
    k2 = _encoder_block_kinet(o1, 64)  # KINET ENCODER

    u2 = _crfb(s2, k2, 64, 0.0625)  # CRFB U2 UNET
    o2 = _crfb(k2, s2, 64, 16)  # CRFB O2 KINET

    s3 = _encoder_block_unet(u2, 128)  # UNET ENCODER
    k3 = _encoder_block_kinet(o2, 128)  # KINET ENCODER

    u3 = _crfb(s3, k3, 128, 0.015625)  # CRFB U3 UNET
    o3 = _crfb(k3, s2, 128, 32)  # CRFB O3 KINET

    # DECODER BLOCK #

    d1_u = _decoder_block_unet(u3, 64)  # UNET DECODER
    d1_k = _decoder_block_kinet(o3, 64)  # KINET DECODER

    d_u1 = _crfb(d1_u, d1_k, 32, 0.0625)  # CRFB D_U1 UNET
    d_o1 = _crfb(d1_k, d1_u, 32, 16)  # CRFB D_O1 KINET

    out = layers.Concatenate()([d_u1, s2])  # CONCATENTATION D_U1 & S2 UNET
    out1 = layers.Concatenate()([d_o1, k2])  # CONCATENTATION D_O1 & K2 KINET

    d2_u = _decoder_block_unet(out, 32)  # UNET DECODER
    d2_k = _decoder_block_kinet(out1, 32)  # KINET DECODER

    d_u2 = _crfb(d2_u, d2_k, 16, 0.25)  # CRFB D_U2 UNET
    d_o2 = _crfb(d2_k, d2_u, 16, 4)  # CRFB D_O2 KINET

    out = layers.Concatenate()([d_u2, s1])  # CONCATENATION D_U2 & S1 UNET
    out1 = layers.Concatenate()([d_o2, k1])  # CONCATENATION D_O2 & K2 KINET

    d3_u = _decoder_block_unet(out, 16)  # UNET DECODER
    d3_k = _decoder_block_kinet(out1, 16)  # KINET DECODER

    out = layers.Concatenate()([d3_u, d3_k])  # FINAL CONCATENATION OUTPUT FROM UNET AND KINET

    out = layers.Conv2D(3, 3, padding="same", activation="relu")(out)  # FINAL CONVOLUTIONAL LAYER
    kiunet_model = keras.Model(inputs, out, name="U-Net")  # MODEL

    return kiunet_model

In [7]:
def resizeImg(img, out_size):
    if img.endswith(".png"):
        # Get original image dimensions
        original_image = sitk.ReadImage(img, imageIO="PNGImageIO")
    elif img.endswith(".tif"):
        original_image = sitk.ReadImage(img, imageIO="TIFFImageIO")
    orig_size = original_image.GetSize()
    orig_spacing = original_image.GetSpacing()
    origin = original_image.GetOrigin()
    direction = original_image.GetDirection()

    # Calculate output spacing
    out_spacing = [
        int(np.round(orig_spacing[0] * (orig_size[0] / out_size[0]))),
        int(np.round(orig_spacing[1] * (orig_size[1] / out_size[1])))
        # might have to put RGB channel here?
    ]

    # Set output dimensions
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputOrigin(origin)
    resampler.SetOutputDirection(direction)
    resampler.SetSize(out_size)
    resampler.SetOutputSpacing(out_spacing)

    return resampler.Execute(original_image)


# Pre-process training dataset
image_num = 1

for img in os.listdir("data/training/av"):
    if img.endswith(".png") or img.endswith(".tif"):
        img = "data/training/av/" + img
        # png_img = Image.open(os.path.join("./training/av", img))
        new_img = resizeImg(img, (128, 128))
        path = "data/training/av/processed_labels/{}_label_downsampled.png".format(image_num)
        sitk.WriteImage(new_img, path)
        image_num += 1

image_num = 1

for img in os.listdir("data/training/images"):
    if img.endswith(".png") or img.endswith(".tif"):
        img = "data/training/images/" + img
        new_img = resizeImg(img, (128, 128))
        path = "data/training/images/processed_labels/{}_img_downsampled.tif".format(image_num)
        sitk.WriteImage(new_img, path)
        image_num += 1

# Pre-process test dataset
image_num = 1

for img in os.listdir("data/testing/av"):
    if img.endswith(".png") or img.endswith(".tif"):
        img = "data/testing/av/" + img
        # png_img = Image.open(os.path.join("./training/av", img))
        new_img = resizeImg(img, (128, 128))
        path = "data/testing/av/processed_labels/{}_label_downsampled.png".format(image_num)
        sitk.WriteImage(new_img, path)
        image_num += 1

image_num = 1

for img in os.listdir("data/testing/images"):
    if img.endswith(".png") or img.endswith(".tif"):
        img = "data/testing/images/" + img
        new_img = resizeImg(img, (128, 128))
        path = "data/testing/images/processed_labels/{}_img_downsampled.tif".format(image_num)
        sitk.WriteImage(new_img, path)
        image_num += 1

In [8]:
x_train = import_data_numpy("data/training/images/processed_labels")
y_train = import_data_numpy("data/training/av/processed_labels")

x_test = import_data_numpy("data/testing/images/processed_labels")
y_test = import_data_numpy("data/testing/av/processed_labels")

In [9]:
kiunet_model = kiunet((128,128,3))

# Save the model structure
kiunet_model.save('kiUnet_structure')

kiunet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss="binary_crossentropy",
    metrics=[keras.metrics.MeanIoU(num_classes=500)])#check metric

2021-10-16 14:48:27.446962: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-16 14:48:27.447975: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-16 14:48:27.451350: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>
<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>
<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>


2021-10-16 14:48:29.343202: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: kiUnet_structure/assets


In [10]:
kiunet_model.fit(x_train, y_train, batch_size=1, epochs=400, 
                 validation_data=(x_test, y_test), verbose=1, 
                 callbacks=keras.callbacks.TensorBoard(log_dir="./logs"))    ## log metrics in TensorBoard)

2021-10-16 14:48:32.014964: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-10-16 14:48:32.015007: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-10-16 14:48:32.015059: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-10-16 14:48:32.067144: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-16 14:48:32.067769: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294635000 Hz


Epoch 1/400
 1/20 [>.............................] - ETA: 3:23 - loss: -40.6707 - mean_io_u: 0.0020

2021-10-16 14:48:42.972539: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-10-16 14:48:42.972601: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


 2/20 [==>...........................] - ETA: 2:09 - loss: -59.4294 - mean_io_u: 0.0016

2021-10-16 14:48:49.978420: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-10-16 14:48:49.995843: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-10-16 14:48:50.038481: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2021_10_16_14_48_49
2021-10-16 14:48:50.053444: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2021_10_16_14_48_49/caclogin03.trace.json.gz
2021-10-16 14:48:50.162180: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2021_10_16_14_48_49
2021-10-16 14:48:50.167107: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to ./logs/train/plugins/profile/2021_10_16_14_48_49/caclogin03.memory_profile.json.gz
2021-10-16 14:48:50.170427:

20/20 [==============================] - 179s 9s/step - loss: -18.0828 - mean_io_u: 0.0014 - val_loss: 111.9623 - val_mean_io_u: 0.1943
Epoch 2/400
20/20 [==============================] - 173s 9s/step - loss: 117.5429 - mean_io_u: 0.4851 - val_loss: 111.9622 - val_mean_io_u: 0.2429
Epoch 3/400
20/20 [==============================] - 172s 9s/step - loss: 121.2999 - mean_io_u: 0.4846 - val_loss: 111.9622 - val_mean_io_u: 0.2429
Epoch 4/400
20/20 [==============================] - 173s 9s/step - loss: 118.1511 - mean_io_u: 0.4850 - val_loss: 111.9622 - val_mean_io_u: 0.2429
Epoch 5/400
20/20 [==============================] - 174s 9s/step - loss: 119.6325 - mean_io_u: 0.4848 - val_loss: 111.9622 - val_mean_io_u: 0.2429
Epoch 6/400
20/20 [==============================] - 173s 9s/step - loss: 115.5048 - mean_io_u: 0.4853 - val_loss: 111.9622 - val_mean_io_u: 0.2429
Epoch 7/400
20/20 [==============================] - 172s 9s/step - loss: 114.5548 - mean_io_u: 0.4854 - val_loss: 111.9622 

In [11]:
# Save the entire model(shoule be with weights) and the weights
kiunet_model.save('kiUnet_model')
kiunet_model.save_weights('kiUnet_saved_weight')

INFO:tensorflow:Assets written to: kiUnet_model/assets
